# Fase 3: Generación Aumentada (RAG) y Evaluación de Modelos

## 1. Introducción
Tras haber completado la **Ingeniería de Datos** (Fase 1: Ingesta y Limpieza) y validado el **Motor de Recuperación** (Fase 2: Indexación en Elasticsearch), entramos en la etapa final y más crítica del sistema: la **Generación**.

En este notebook, desplegaremos la inteligencia del sistema utilizando la GPU disponible (RTX 4090). El objetivo es conectar el "cerebro" (Modelos de Lenguaje) con la "memoria" (Nuestra base de datos de noticias) para sintetizar respuestas veraces y actualizadas.

## 2. Objetivos del Notebook
Siguiendo las directrices del TFG, este entorno está diseñado para ser modular y permitir la comparación experimental entre diferentes arquitecturas. Los pasos técnicos serán:

1.  **Configuración del Entorno de Inferencia:** Inicialización de librerías de Deep Learning (`transformers`, `accelerate`) y verificación de acceso a la GPU.
2.  **Despliegue de Modelos (SLM vs LLM):**
    * Carga de un **Small Language Model (SLM)**: Empezaremos con *Phi-3* (3.8B parámetros) para validar la hipótesis de que un modelo pequeño con buen contexto puede rendir bien.
    * Preparación para **Large Language Models (LLM)**: El código será reutilizable para cargar modelos mayores (como *Llama-3* o *Mistral*) posteriormente.
3.  **Fusión RAG (Retrieval-Augmented Generation):** Implementación de la función `ask_rag(question)` que orquesta el flujo completo:
    * *Input Usuario* $\rightarrow$ *Búsqueda en Elasticsearch* $\rightarrow$ *Construcción del Prompt* $\rightarrow$ *Generación*.
4.  **Ingeniería de Prompts:** Diseño de un "System Prompt" estricto que obligue al modelo a basarse exclusivamente en las noticias recuperadas para mitigar alucinaciones.

## 3. Arquitectura del Flujo
El sistema no permite que el LLM improvise. El flujo de datos es unidireccional y forzado:
> **Pregunta** $\xrightarrow{BM25}$ **Noticias (Contexto)** $\rightarrow$ **[INSTRUCCIÓN + CONTEXTO + PREGUNTA]** $\rightarrow$ **LLM** $\rightarrow$ **Respuesta Verificada**

---

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 1. Configuración de Hardware
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Hardware detectado: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")

# 2. Selección de Modelo (Empezamos con un SLM para la prueba rápida)
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

print(f"Cargando modelo: {MODEL_ID}...")

# 3. Carga del Tokenizer y el Modelo 
# Importante: Ponemos trust_remote_code=False para usar la versión estable instalada
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=False)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",             # Distribuye automáticamente en la GPU
    torch_dtype=torch.float16    # Usamos precisión media para ahorrar VRAM y ganar velocidad
)

# 4. Crear Pipeline de Generación
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("\n🚀 ¡Modelo SLM cargado y listo para inferencia!")

/home/javierruiz/miniconda3/envs/environment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware detectado: NVIDIA GeForce RTX 4090
Cargando modelo: microsoft/Phi-3-mini-4k-instruct...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 195/195 [00:01<00:00, 125.96it/s, Materializing param=model.norm.weight]                              



🚀 ¡Modelo SLM cargado y listo para inferencia!


In [2]:
# 5. Prueba de Inferencia (Sin RAG todavía)
# Vamos a ver si el "cerebro" funciona por sí solo.

messages = [
    {"role": "user", "content": "¿Podrías explicarme brevemente qué es la inflación económica?"},
]

# Configuración de generación
generation_args = {
    "max_new_tokens": 150,     # Que no escriba una novela, solo un párrafo
    "return_full_text": False, # Que no repita la pregunta
    "temperature": 0.1,        # Creatividad baja (queremos datos, no poesía)
    "do_sample": True,
}

print(" Generando respuesta...")
output = pipe(messages, **generation_args)
print("\n" + "="*50)
print(output[0]['generated_text'])
print("="*50)

Passing `generation_config` together with generation-related arguments=({'do_sample', 'temperature', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Generando respuesta...

 La inflación económica es un fenómeno que se refiere a la tasa a la cual el nivel general de precios de bienes y servicios está aumentando, y, por consiguiente, el poder adquisitivo está disminuyendo. En otras palabras, con la inflación, cada unidad de moneda compra menos bienes y servicios. Esto puede deberse a diversos factores, como un aumento en la oferta de dinero, un aumento en los costos de producción, o una disminución en la demanda de bienes y servicios. La inflación es un indicador importante de la salud económica de


In [3]:
from elasticsearch import Elasticsearch

# Ahora que las versiones coinciden, la configuración es estándar y sencilla
es = Elasticsearch("http://127.0.0.1:9250")

try:
    if es.ping():
        print("✅ PING EXITOSO: Versiones sincronizadas (8.12.0).")
        print(f"   Conectado a: {es.info()['name']}")
    else:
        print("❌ El servidor no responde. ¿Está el túnel abierto?")
except Exception as e:
    print(f"❌ Error inesperado: {e}")

✅ PING EXITOSO: Versiones sincronizadas (8.12.0).
   Conectado a: valencia


In [7]:
def ask_rag(query, top_k=1): # <--- Forzamos k=1 como pide el profesor
    """
    Función RAG para TFG: Recupera 1 noticia y genera respuesta basada estrictamente en ella.
    """
    # --- A. BÚSQUEDA (Retrieval de 1 sola noticia) ---
    search_payload = {
        "size": top_k,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^5", "body"], # Mucho peso al título para el 'match perfecto'
                "fuzziness": 0 # 0 = Match perfecto (sin errores tipográficos permitidos)
            }
        },
        "_source": ["title", "body", "date", "source"]
    }
    
    try:
        response = es.search(index="noticias_tfg", body=search_payload)
        hits = response['hits']['hits']
    except Exception as e:
        return f"Error en Elasticsearch: {e}"
    
    if not hits:
        return "⚠️ NO ENCONTRADO: No hay ninguna noticia que coincida exactamente con la búsqueda."

    # --- B. EXTRACCIÓN DE LA NOTICIA ÚNICA ---
    doc = hits[0]['_source']
    contexto_unico = f"""
    TITULO: {doc.get('title')}
    FECHA: {doc.get('date', 'Desconocida')}
    FUENTE: {doc.get('source', 'Desconocida')}
    CONTENIDO: {doc.get('body')}
    """

    # --- C. PROMPT PARA MITIGAR ALUCINACIONES ---
    messages = [
        {"role": "user", "content": f"""
        Eres un sistema de verificación de datos (Fact-Checking). 
        Tu objetivo es responder a la pregunta usando ÚNICAMENTE el texto que te proporciono abajo.

        REGLAS CRÍTICAS:
        1. Si la respuesta no aparece en el texto, responde exactamente: "No tengo información suficiente en mis archivos".
        2. No utilices conocimiento externo.
        3. No menciones otras noticias que no sean la proporcionada.

        ### TEXTO DE REFERENCIA:
        {contexto_unico}
        
        ### PREGUNTA:
        {query}
        """}
    ]

    # --- D. GENERACIÓN ---
    outputs = pipe(
        messages,
        max_new_tokens=256,
        do_sample=False, # Ponemos False para que sea determinista (siempre responda igual)
        temperature=0.0, # Temperatura 0 para evitar inventos
    )
    
    return outputs[0]['generated_text']

print(" Sistema RAG (k=1) configurado para match perfecto.")

 Sistema RAG (k=1) configurado para match perfecto.


In [8]:
pregunta_tfg = "¿Qué aranceles puso Trump a China?"

# 1. Prueba SIN RAG 
print("--- RESPUESTA SLM (SIN RAG) ---")
res_base = pipe([{"role": "user", "content": pregunta_tfg}], max_new_tokens=150)
print(res_base[0]['generated_text'])

print("\n" + "-"*40 + "\n")

# 2. Prueba CON RAG 
print("--- RESPUESTA SISTEMA RAG (k=1) ---")
res_rag = ask_rag(pregunta_tfg)
print(res_rag)

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--- RESPUESTA SLM (SIN RAG) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'role': 'user', 'content': '¿Qué aranceles puso Trump a China?'}, {'role': 'assistant', 'content': ' El presidente Donald Trump impuso aranceles a China como parte de la Ley de Protección de la Industria Automotriz (Auto Industry Trade Policy Act), conocida simplemente como "Aranceles a la China". Estos aranceles se enmarcaron en el contexto de una larga disputa comercial entre Estados Unidos y China que buscaba proteger la industria automotriz frente a la competencia china. La implementación de estos aranceles, junto con otros sanciones comerciales, fue un punto de inflexión en las relaciones económicas entre los dos países.'}]

----------------------------------------

--- RESPUESTA SISTEMA RAG (k=1) ---
[{'role': 'user', 'content': '\n        Eres un sistema de verificación de datos (Fact-Checking). \n        Tu objetivo es responder a la pregunta usando ÚNICAMENTE el texto que te proporciono abajo.\n\n        REGLAS CRÍTICAS:\n        1. Si la respuesta no aparece en el texto, re